<a href="https://colab.research.google.com/github/VietHoangg03/Machine_Learning_Basic/blob/main/FireBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Firebase:
- Hỗi trợ lưu trữ dữ liệu dạng JSON tree format
- Hỗi trợ dữ liệu real-time data cả hai clients. Cơ chế synchronization cơ chế đồng bộ.
- Cấu trúc đơn giản để triển khai (deloy) nhưng thì không hỗi trợ query phức tạp.
=> Đây là fire base Realtime.

Firebase Cloud Firestore:
- Lưu trữ dạng Document giống MongoDB, hỗi trợ queri phức tạp realtime cũng có sẵn nhưng chưa được tối ưu.
- Hoạt động cơ chế snapshot. cơ chế copy dữ liệu
=> Tuỳ thuộc nhu cầu để chọn lựa 2 kiểu dữ liệu.

Giới hạn của Firebase Realtime:
- Không hỗi trợ truy vấn dữ liệu phức tạp.
- Nếu data bị lỗi thì khó để sửa.

Giới hạn của Firebase Cloud:
- Viết Document 1 lần 1 giây
- Giới hạn 20,000 trường trong 1 document
- Mỗi document giới hạn 1 MB
- 1G lưu trữ miễn phí và 50000 lượt viết 1 ngày, 20000 lượt đọc 1 ngày.

Tổ chức database:
- Collection
  + Document
      * Data (Json...)

Muốn tổ chức chỉnh tru thì dử dụng Firebase Cloud Firestore.



In [9]:
!pip install firebase-admin

In [3]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("/content/aio2025-6bb26-firebase-adminsdk-fbsvc-7d8039b6cf.json")

check =firebase_admin.initialize_app(cred)
if check:
 print("Đã kết nối thành công")

Đã kết nối thành công


In [7]:
from firebase_admin import firestore

firestore_db = firestore.client()

if firestore_db:
  print("Kết nối thành công")

Kết nối thành công


Ghi dữ liệu vào DB

In [11]:
# Tạo collection 'products'
products_ref = firestore_db.collection('products')


# Thêm document với ID tự động
doc_ref = products_ref.add({
    'name': 'iPhone 15',
    'price': 999,
    'category': 'smartphone',
    'in_stock': True,
    'tags': ['apple', 'ios', 'phone']
})
print(f"✅ Đã thêm sản phẩm với ID: {doc_ref[1].id}")

# Thêm document với ID cụ thể
products_ref.document('product_001').set({
    'name': 'Samsung Galaxy S24',
    'price': 899,
    'category': 'smartphone',
    'in_stock': True,
    'tags': ['samsung', 'android', 'phone']
})
print("✅ Đã thêm sản phẩm với ID: product_001")

✅ Đã thêm sản phẩm với ID: wEXW4QBhIX2Ai0O0Z7Oa
✅ Đã thêm sản phẩm với ID: product_001


In [12]:
docs = products_ref.stream()
print("Tất cả sản phẩm:")
for doc in docs:
    print(f"  {doc.id}: {doc.to_dict()}")


Tất cả sản phẩm:
  product_001: {'price': 899, 'name': 'Samsung Galaxy S24', 'tags': ['samsung', 'android', 'phone'], 'in_stock': True, 'category': 'smartphone'}
  wEXW4QBhIX2Ai0O0Z7Oa: {'price': 999, 'name': 'iPhone 15', 'tags': ['apple', 'ios', 'phone'], 'in_stock': True, 'category': 'smartphone'}


In [13]:
# Đọc một document cụ thể
doc = products_ref.document('product_001').get()
if doc.exists:
    print(f"\nSản phẩm product_001: {doc.to_dict()}")


Sản phẩm product_001: {'price': 899, 'name': 'Samsung Galaxy S24', 'tags': ['samsung', 'android', 'phone'], 'in_stock': True, 'category': 'smartphone'}


Query dữ liệu

In [14]:
# Tìm sản phẩm có giá > 900
expensive_products = products_ref.where('price', '>', 900).stream()
print("Sản phẩm giá > 900:")
for doc in expensive_products:
    print(f"  {doc.id}: {doc.to_dict()}")

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Sản phẩm giá > 900:
  wEXW4QBhIX2Ai0O0Z7Oa: {'price': 999, 'name': 'iPhone 15', 'tags': ['apple', 'ios', 'phone'], 'in_stock': True, 'category': 'smartphone'}


In [15]:
# Tìm sản phẩm theo category
smartphones = products_ref.where('category', '==', 'smartphone').stream()
print("\nSmartphones:")
for doc in smartphones:
    print(f"  {doc.id}: {doc.to_dict()}")


Smartphones:
  product_001: {'price': 899, 'name': 'Samsung Galaxy S24', 'tags': ['samsung', 'android', 'phone'], 'in_stock': True, 'category': 'smartphone'}
  wEXW4QBhIX2Ai0O0Z7Oa: {'price': 999, 'name': 'iPhone 15', 'tags': ['apple', 'ios', 'phone'], 'in_stock': True, 'category': 'smartphone'}


In [16]:
# 4. Cập nhật dữ liệu
print("\n4. Cập nhật dữ liệu:")
products_ref.document('product_001').update({
    'price': 849,
    'last_updated': firestore.SERVER_TIMESTAMP
})
print("✅ Đã cập nhật giá sản phẩm product_001")


4. Cập nhật dữ liệu:
✅ Đã cập nhật giá sản phẩm product_001


In [17]:
print("\n5. Tạo test:")
test_doc_ref = products_ref.document('test_product')
test_doc_ref.set({'name': 'Test Product', 'temp': True})
print("✅ Đã tạo test_product")


5. Tạo test:
✅ Đã tạo test_product


In [18]:
products_ref.document('test_product').delete()
print("✅ Đã xóa test_product")

✅ Đã xóa test_product


In [20]:
# Truy vấn phức tạp

query = firestore_db.collection('products')\
    .where('in_stock', '==', True)\
    .where('category', '==', 'smartphone')\
    .where('tags', 'array_contains', 'phone')\
    .where('price', '<', 900)

results = query.stream()

for doc in results:
    print(f"{doc.id} => {doc.to_dict()}")

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/tmp/ipython-input-20-1671042940.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('category', '==', 'smartphone')\
/tmp/ipython-input-20-1671042940.py:6: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('tags', 'array_contains', 'phone')\
/tmp/ipython-input-20-1671042940.py:7: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('price', '<', 900)


product_001 => {'name': 'Samsung Galaxy S24', 'price': 849, 'tags': ['samsung', 'android', 'phone'], 'in_stock': True, 'last_updated': DatetimeWithNanoseconds(2025, 7, 17, 13, 51, 25, 135000, tzinfo=datetime.timezone.utc), 'category': 'smartphone'}


In [21]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [22]:
# Tải iris dataset
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df['target_name'] = iris_df['target'].map(lambda x: iris.target_names[x])

print("Iris dataset loaded successfully!")
print(f"Dataset shape: {iris_df.shape}")
print("\nFirst 5 rows:")
print(iris_df.head())

Iris dataset loaded successfully!
Dataset shape: (150, 6)

First 5 rows:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target target_name  
0       0      setosa  
1       0      setosa  
2       0      setosa  
3       0      setosa  
4       0      setosa  


In [23]:
db = firestore.client()

In [24]:
# Upload dữ liệu lên Firestore
def upload_iris_to_firestore():
    collection_name = 'iris_dataset'

    # Xóa collection cũ nếu tồn tại (tùy chọn)
    # docs = db.collection(collection_name).stream()
    # for doc in docs:
    #     doc.reference.delete()

    batch_size = 100
    total_records = len(iris_df)

    for i in range(0, total_records, batch_size):
        batch = db.batch()
        batch_data = iris_df.iloc[i:i+batch_size]

        for idx, row in batch_data.iterrows():
            doc_ref = db.collection(collection_name).document(f'sample_{idx}')

            # Chuyển đổi numpy types sang Python native types
            data = {
                'sepal_length': float(row['sepal length (cm)']),
                'sepal_width': float(row['sepal width (cm)']),
                'petal_length': float(row['petal length (cm)']),
                'petal_width': float(row['petal width (cm)']),
                'species': str(row['target_name']),
            }

            batch.set(doc_ref, data)

        # Commit batch
        batch.commit()
        print(f"Uploaded batch {i//batch_size + 1}: records {i+1}-{min(i+batch_size, total_records)}")

    print(f"\nSuccessfully uploaded {total_records} records to Firestore collection '{collection_name}'")

# Thực hiện upload
try:
    upload_iris_to_firestore()
    print("\n✅ Upload completed successfully!")
except Exception as e:
    print(f"❌ Error during upload: {str(e)}")

# Kiểm tra dữ liệu đã upload
def verify_upload():
    collection_name = 'iris_dataset'
    docs = db.collection(collection_name).limit(5).stream()

    print("\nVerifying upload - First 5 documents:")
    for doc in docs:
        print(f"Document ID: {doc.id}")
        print(f"Data: {doc.to_dict()}")
        print("-" * 50)

# Chạy kiểm tra
verify_upload()

# Thống kê dữ liệu đã upload
def get_collection_stats():
    collection_name = 'iris_dataset'
    docs = list(db.collection(collection_name).stream())

    print(f"\nCollection Statistics:")
    print(f"Total documents: {len(docs)}")

    # Đếm theo target_name
    target_counts = {}
    for doc in docs:
        data = doc.to_dict()
        target_name = data.get('target_name', 'unknown')
        target_counts[target_name] = target_counts.get(target_name, 0) + 1

    print(f"Distribution by species:")
    for species, count in target_counts.items():
        print(f"  {species}: {count}")

# Chạy thống kê
get_collection_stats()

Uploaded batch 1: records 1-100
Uploaded batch 2: records 101-150

Successfully uploaded 150 records to Firestore collection 'iris_dataset'

✅ Upload completed successfully!

Verifying upload - First 5 documents:
Document ID: sample_0
Data: {'species': 'setosa', 'petal_width': 0.2, 'sepal_width': 3.5, 'petal_length': 1.4, 'sepal_length': 5.1}
--------------------------------------------------
Document ID: sample_1
Data: {'species': 'setosa', 'petal_width': 0.2, 'sepal_width': 3.0, 'petal_length': 1.4, 'sepal_length': 4.9}
--------------------------------------------------
Document ID: sample_10
Data: {'species': 'setosa', 'petal_width': 0.2, 'sepal_width': 3.7, 'petal_length': 1.5, 'sepal_length': 5.4}
--------------------------------------------------
Document ID: sample_100
Data: {'species': 'virginica', 'petal_width': 2.5, 'sepal_width': 3.3, 'petal_length': 6.0, 'sepal_length': 6.3}
--------------------------------------------------
Document ID: sample_101
Data: {'species': 'virgin